In [1]:
from qdrant_client import QdrantClient, models
import os

client = QdrantClient(':memory:')

In [2]:

collection_name = "store"
vector_size = 768

if client.collection_exists(collection_name=collection_name):
    client.delete_collection(collection_name=collection_name)

client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=vector_size,
        distance=models.Distance.COSINE,
    ),
    optimizers_config=models.OptimizersConfigDiff(
        indexing_threshold=100,
    ),
)

# Index frequently filtered fields
client.create_payload_index(
    collection_name=collection_name,
    field_name="category",
    field_schema=models.PayloadSchemaType.KEYWORD,
)

client.create_payload_index(
    collection_name=collection_name,
    field_name="price",
    field_schema=models.PayloadSchemaType.FLOAT,
)

client.create_payload_index(
    collection_name=collection_name,
    field_name="brand",
    field_schema=models.PayloadSchemaType.KEYWORD,
)

/tmp/ipykernel_48280/109709024.py:19: UserWarning: Payload indexes have no effect in the local Qdrant. Please use server Qdrant if you need payload indexes.
  client.create_payload_index(


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [3]:
# Upload data
import random

points = []
for i in range(1000):
    points.append(
        models.PointStruct(
            id=i,
            vector=[random.random() for _ in range(vector_size)],
            payload={
                "category": random.choice(["laptop", "phone", "tablet"]),
                "price": random.randint(0, 1000),
                "brand": random.choice(
                    ["Apple", "Dell", "HP", "Lenovo", "Asus", "Acer", "Samsung"]
                ),
            },
        )
    )
client.upload_points(
    collection_name=collection_name,
    points=points,
)

In [ ]:
# Create filter combining multiple conditions
filter_conditions = models.Filter(
    must=[
        models.FieldCondition(key="category", match=models.MatchValue(value="laptop")),
        models.FieldCondition(key="price", range=models.Range(lte=1000)),
        models.FieldCondition(key="brand", match=models.MatchAny(any=["Apple", "Dell", "HP"])),
    ]
)

query_vector = [random.random() for _ in range(vector_size)]

# Execute filtered search
results = client.query_points(
    collection_name=collection_name,
    query=query_vector,
    query_filter=filter_conditions,
    limit=10,
    search_params=models.SearchParams(hnsw_ef=128),
)

In [15]:
for i, point in enumerate(results.points, 1):
    payload = point.payload
    print(
        f"{i}. Category: {payload['category']} \nPrice: {payload['price']}$ \nBrand: {payload['brand']}\n"
        f"Score: {point.score:.4f}\n"
    )

1. Category: laptop 
Price: 183$ 
Brand: Apple
Score: 0.7804

2. Category: laptop 
Price: 23$ 
Brand: Dell
Score: 0.7793

3. Category: laptop 
Price: 463$ 
Brand: Apple
Score: 0.7761

4. Category: laptop 
Price: 281$ 
Brand: HP
Score: 0.7759

5. Category: laptop 
Price: 381$ 
Brand: HP
Score: 0.7750

6. Category: laptop 
Price: 477$ 
Brand: Apple
Score: 0.7750

7. Category: laptop 
Price: 759$ 
Brand: Dell
Score: 0.7744

8. Category: laptop 
Price: 63$ 
Brand: HP
Score: 0.7735

9. Category: laptop 
Price: 63$ 
Brand: HP
Score: 0.7729

10. Category: laptop 
Price: 484$ 
Brand: Apple
Score: 0.7719

